# Data Story

In [ ]:
import sys
import os

database_path = '../database'

print(sys.path)

def import_system_path():
    if database_path not in sys.path:
        sys.path.append(database_path)
        print('New path added to Sys.')
    else:
        print('Path already exists.')
        
import_system_path()

In [1]:
import numpy as np
import pandas as pd

import re

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from DBA import DBA

In [2]:
xlsx = pd.ExcelFile('data_collectionklimadaten_swiss_open_data.xlsx')
swiss_sunshine = pd.read_excel(xlsx, 'Sonnenscheindauer' ,
                                   header=5, index_col=0 ).dropna().iloc[1:]

swiss_precipitation = pd.read_excel(xlsx, 'Jahresniederschlag' ,
                                         header=5, index_col=0 ).dropna().iloc[1:]

swiss_temp = pd.read_excel(xlsx, 'Jahrestemperatur' ,
                                header=5, index_col=0 ).dropna().iloc[1:]

swiss_snow = pd.read_excel(xlsx, 'Neuschnee' ,
                           header=5, index_col=0 ).dropna().iloc[1:]


co2_data = pd.read_csv('co2.csv', index_col=1).drop(['Unnamed: 0'], axis=1)
co2_data.index = pd.to_datetime(co2_data.index)

temp_data = pd.read_csv('temperature_global.csv', index_col=1).drop(['Unnamed: 0'], axis=1)
temp_data.index = pd.to_datetime(temp_data.index)

In [3]:
def swiss_places_column_titles(data):
    """Replaces the Header of the swiss data table with only alphabetic chars."""
    new_cols = []
    
    for i in data.columns:
        result = re.sub("[1,2) ]", '', i)
        new_cols.append(result)
    data.columns = new_cols
    
    return data  

In [4]:
swiss_sunshine = swiss_places_column_titles(swiss_sunshine)
swiss_precipitation = swiss_places_column_titles(swiss_precipitation)
swiss_snow = swiss_places_column_titles(swiss_snow)
swiss_temp = swiss_places_column_titles(swiss_temp)

Jan geht jeden Winter  zu seiner Grossmutter in die Skiferien nach Davos. Seine Grossmutter lebt bereits ihr ganzes Leben im Davoser-Dorf. Immer wieder tönt sie an, es gäbe von Jahr zu Jahr immer weniger Schnee im Dorf und auch im Skigebiet. Hat sich seine Grossmutter das nur ausgedacht oder steckt da wirklich einen Funken Wahrheit dahinter.
Jan ist sehr faktenorientiert und möchte sich dies einmal genauer ansehen. Er recherchiert...

Jan findet eine Infografik über die jährliche gemessene Schneehöhe in Davos-Klosters.

In [12]:
def lineplot_davos_snow(data):
    
    data = data['DavosWSL']
    new_val = np.where(data == '...' ,None , data*1)
    data = pd.DataFrame(new_val, index=data.index).dropna().astype('int')
    
    data = data - data.iloc[0]
    data.loc[:,'color'] = np.where(data >= 0, 1, 0)
    data.columns = ['snow', 'color']
    data.index = pd.to_numeric(data.index)
  
    fig = px.bar(x=data.index, 
                 y=data['snow'], 
                 color=data['color'], 
                 title='Jährliche Abweichungen Neuschnee zu {} in Davos von {} bis {}'.format(data.index[0], data.index[0], data.index[-1]),
                 labels={'x':'Jahre', 'y':'Abweichung Neuschnee pro Jahr (cm)'},  
                 template='simple_white',
                 color_continuous_scale=['red', 'blue'],
                 opacity=0.7
            )
    
    fig.update(layout_coloraxis_showscale=False)
    fig.show()
     
lineplot_davos_snow(swiss_snow)

In [33]:
def lineplot_davos_snow(data):
    
    data = data['DavosWSL']
    new_val = np.where(data == '...' ,None , data*1)
    data = pd.DataFrame(new_val, index=data.index).dropna().astype('int')
    
    data = data - data.iloc[0]
    data.loc[:,'color'] = np.where(data >= 0, 1, 0)
    data.columns = ['snow', 'color']
    data.index = pd.to_numeric(data.index)
  
    fig = px.scatter(x=data.index, 
                     y=data['snow'], 
                     color=data['color'], 
                     trendline='ols',
                     title='Jährliche Abweichungen Neuschnee zu {} in Davos von {} bis {}'.format(data.index[0], data.index[0], data.index[-1]),
                     labels={'x':'Jahre', 'y':'Abweichung Neuschnee pro Jahr (cm)'},  
                     template='simple_white'  
                     )
    
    fig.update_traces(mode='lines')
    fig.update(layout_coloraxis_showscale=False)
    fig.show()
     
lineplot_davos_snow(swiss_snow)

Jan verfolgt den Trend und merkt, dass es seit 1960 immer weniger schneereiche Jahre gibt. Er möchte das prüft den Trend in dem er eine Lineare Regression über den Trend legt. Der Trend der Regressionslinie zeigt nach unten, was darauf hindeutet, dass es wirklich immer weniger Schnee in Davos gibt. Er findet eine weitere Grafik, welche die jährliche Durchschnittstemperatur präsentiert. 

In [15]:
from sklearn.linear_model import LinearRegression

def lineplot_swiss_temp(data):
    
    data = data['Davos']

    data = data.loc[:] - data.loc[data.index[0]]
    
    data = data.reset_index()
    data.columns = ['year', 'temperature']
    
    fig = px.scatter(x=data['year'], y=data['temperature'])
    
    fig.update_traces(mode='lines')
    fig.show()
    
    
lineplot_swiss_temp(swiss_temp)